In [2]:
from bioimage import BioImageFile

In [3]:
f = BioImageFile("test.nd2").run()

1
1


In [5]:
f.meta.allmetadata

[<Image ID="Image:0" Name="test.nd2 (series 1)"><InstrumentRef ID="Instrument:0"/><ObjectiveSettings ID="Objective:0:0" RefractiveIndex="1.515"/><Pixels BigEndian="false" DimensionOrder="XYCZT" ID="Pixels:0" Interleaved="false" PhysicalSizeX="0.2071601898007465" PhysicalSizeXUnit="µm" PhysicalSizeY="0.2071601898007465" PhysicalSizeYUnit="µm" PhysicalSizeZ="0.5" PhysicalSizeZUnit="µm" SignificantBits="12" SizeC="2" SizeT="1" SizeX="1024" SizeY="1024" SizeZ="35" Type="uint16"><Channel Color="16711935" EmissionWavelength="535.0" EmissionWavelengthUnit="nm" ID="Channel:0:0" Name="Alexa 488 antibody" SamplesPerPixel="1"><DetectorSettings ID="Detector:0:0"/><LightPath/></Channel><Channel Color="65535" EmissionWavelength="825.0" EmissionWavelengthUnit="nm" ID="Channel:0:1" Name="Alx647" SamplesPerPixel="1"><DetectorSettings ID="Detector:0:0"/><LightPath/></Channel><MetadataOnly/><Plane DeltaT="0.7572736046038335" DeltaTUnit="s" ExposureTime="7845.979283975437" ExposureTimeUnit="s" PositionZ="

In [7]:
f.getZStack().shape

(1024, 1024, 3, 35)